## Loading libraries

In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import time
import lightgbm as lgbm
from pathlib import Path
import pickle
from catboost import CatBoostClassifier, cv, Pool
import scikitplot as skplt
from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, tpe, partial

import itertools
from itertools import combinations

import scipy as sp
from scipy.stats import pearsonr, chi2_contingency

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap

from datetime import datetime
from dateutil import relativedelta

from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None
pd.options.display.max_rows = None

import statsmodels.api as sm 
from statsmodels.graphics.api import abline_plot # For visualling evaluating predictions.
from statsmodels.stats.proportion import proportion_confint

import warnings # For handling error messages.
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.filterwarnings('ignore')

import sklearn.metrics as met
from sklearn import linear_model, preprocessing, model_selection, svm, datasets
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2, RFE
from sklearn.linear_model import LassoCV, LogisticRegression, Lasso
from sklearn.metrics import plot_confusion_matrix, auc, confusion_matrix, classification_report, accuracy_score, roc_curve, roc_auc_score, plot_roc_curve
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import scale, StandardScaler, LabelEncoder, MinMaxScaler, Binarizer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs

## Loading dataset

In [2]:
df = pd.read_csv('df1.csv', index_col='id') 

### First looks.

In [3]:
df.head()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=68407277,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=68355089,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.0,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,

In [4]:
df.tail()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
89905081,NaN,18000.0,18000.0,18000.0,60 months,9.49,377.95,B,B2,NaN,5 years,OWN,130000.0,Not Verified,Oct-2016,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=89905081,NaN,home_improvement,Home improvement,775xx,TX,20.59,0.0,Jul-2004,735.0,739.0,1.0,NaN,NaN,17.0,0.0,23833.0,34.0,39.0,f,0.0,0.0,20756.233632,20756.23,18000.00,2756.23,0.0,0.0,0.0,Aug-2018,12828.77,NaN,Mar-2019,709.0,705.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,168758.0,0.0,6.0,0.0,3.0,15.0,30516.0,52.0,0.0,3.0,7061.0,41.0,70100.0,1.0,2.0,1.0,6.0,9927.0,4161.0,68.2,0.0,0.0,113.0,146.0,17.0,15.0,3.0,17.0,NaN,2.0,NaN,0.0,3.0,9.0,3.0,5.0,21.0,10.0,15.0,9.0,17.0,0.0,0.0,0.0,0.0,100.0,33.3,0.0,0.0,275356.0,54349.0,13100.0,77756.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
88948836,NaN,29400.0,29400.0,29400.0,60 months,13.99,683.94,C,C3,Chief Operating Officer,9 years,MORTGAGE,180792.0,Not Verified,Oct-2016,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=88948836,NaN,debt_consolidation,Debt consolidation,900xx,CA,22.03,0.0,Mar-2002,705.0,709.0,1.0,NaN,NaN,16.0,0.0,77480.0,85.2,32.0,f,0.0,0.0,35848.764532,35848.76,29400.00,6448.76,0.0,0.0,0.0,Jul-2018,22192.81,NaN,Jul-2018,719.0,715.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,663465.0,0.0,4.0,2.0,4.0,8.0,70652.0,50.0,0.0,0.0,31069.0,68.0,86500.0,3.0,0.0,

### 1.3M rows and 151 columns (including index).

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1345310 entries, 68407277 to 88224441
Columns: 150 entries, member_id to settlement_term
dtypes: float64(114), object(36)
memory usage: 1.5+ GB


### Though it's no longer available on Kaggle, Wendy Kan provided a list of feature descriptions.

acc_now_delinq	The number of accounts on which the borrower is now delinquent

acc_open_past_24mths	Number of trades opened in past 24 months

addr_state	The state provided by the borrower in the loan application

all_util	Balance to credit limit on all trades

annual_inc	The self-reported annual income provided by the borrower during registration.

annual_inc_joint	The combined self-reported annual income provided by the co-borrowers during registration

application_type	Indicates whether the loan is an individual application or a joint application with two co-borrowers

avg_cur_bal	Average current balance of all accounts

bc_open_to_buy	Total open to buy on revolving bankcards

bc_util	Ratio of total current balance to high credit/credit limit for all bankcard accounts

chargeoff_within_12_mths	Number of charge-offs within 12 months

collection_recovery_fee	post charge off collection fee

collections_12_mths_ex_med	Number of collections in 12 months excluding medical collections

debt_settlement_flag	Flags whether or not the borrower, who has charged-off, is working with a debt-settlement company

debt_settlement_flag_date	The most recent date that the Debt_Settlement_Flag has been set  

deferral_term	Amount of months that the borrower is expected to pay less than the contractual monthly payment amount due to a hardship plan

delinq_2yrs	The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years

delinq_amnt	The past-due amount owed for the accounts on which the borrower is now delinquent

desc	Loan description provided by the borrower

disbursement_method	The method by which the borrower receives their loan. Possible values are: CASH, DIRECT_PAY

dti	A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income

dti_joint	A ratio calculated using the co-borrowers' total monthly payments on the total debt obligations, excluding mortgages and the requested LC loan, divided by the co-borrowers' combined self-reported monthly income

earliest_cr_line	The month the borrower's earliest reported credit line was opened

emp_length	Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years

emp_title	The job title supplied by the Borrower when applying for the loan

fico_range_high	The upper boundary range the borrower’s FICO at loan origination belongs to

fico_range_low	The lower boundary range the borrower’s FICO at loan origination belongs to

funded_amnt	The total amount committed to that loan at that point in time

funded_amnt_inv	The total amount committed by investors for that loan at that point in time

grade	LC assigned loan grade

hardship_amount	The interest payment that the borrower has committed to make each month while they are on a hardship plan

hardship_dpd	Account days past due as of the hardship plan start date

hardship_end_date	The end date of the hardship plan period

hardship_flag	Flags whether or not the borrower is on a hardship plan

hardship_last_payment_amount	The last payment amount as of the hardship plan start date

hardship_length	The number of months the borrower will make smaller payments than normally obligated due to a hardship plan

hardship_loan_status	Loan Status as of the hardship plan start date

hardship_payoff_balance_amount	The payoff balance amount as of the hardship plan start date

hardship_reason	Describes the reason the hardship plan was offered

hardship_start_date	The start date of the hardship plan period

hardship_status	Describes if the hardship plan is active, pending, canceled, completed, or broken

hardship_type	Describes the hardship plan offering

home_ownership	The home ownership status provided by the borrower during registration or obtained from the credit report. Our values are: RENT, OWN, MORTGAGE, OTHER

id	A unique LC assigned ID for the loan listing.

il_util	Ratio of total current balance to high credit/credit limit on all install acct

initial_list_status	The initial listing status of the loan. Possible values are – W, F

inq_fi	Number of personal finance inquiries

inq_last_12m	Number of credit inquiries in past 12 months

inq_last_6mths	The number of inquiries in past 6 months (excluding auto and mortgage inquiries)

installment	The monthly payment owed by the borrower if the loan originates.

int_rate	Interest Rate on the loan

issue_d	The month which the loan was funded

last_credit_pull_d	The most recent month LC pulled credit for this loan

last_fico_range_high	The upper boundary range the borrower’s last FICO pulled belongs to.

last_fico_range_low	The lower boundary range the borrower’s last FICO pulled belongs to.

last_pymnt_amnt	Last total payment amount received

last_pymnt_d	Last month payment was received

loan_amnt	The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.

loan_status	Current status of the loan

max_bal_bc	Maximum current balance owed on all revolving accounts

member_id	A unique LC assigned Id for the borrower member.

mo_sin_old_il_acct	Months since oldest bank installment account opened

mo_sin_old_rev_tl_op	Months since oldest revolving account opened

mo_sin_rcnt_rev_tl_op	Months since most recent revolving account opened

mo_sin_rcnt_tl	Months since most recent account opened

mort_acc	Number of mortgage accounts.

mths_since_last_delinq	The number of months since the borrower's last delinquency.

mths_since_last_major_derog	Months since most recent 90-day or worse rating

mths_since_last_record	The number of months since the last public record.

mths_since_rcnt_il	Months since most recent installment accounts opened

mths_since_recent_bc	Months since most recent bankcard account opened.

mths_since_recent_bc_dlq	Months since most recent bankcard delinquency

mths_since_recent_inq	Months since most recent inquiry.

mths_since_recent_revol_delinq	Months since most recent revolving delinquency.

next_pymnt_d	Next scheduled payment date

num_accts_ever_120_pd	Number of accounts ever 120 or more days past due

num_actv_bc_tl	Number of currently active bankcard accounts

num_actv_rev_tl	Number of currently active revolving trades

num_bc_sats	Number of satisfactory bankcard accounts

num_bc_tl	Number of bankcard accounts

num_il_tl	Number of installment accounts

num_op_rev_tl	Number of open revolving accounts

num_rev_accts	Number of revolving accounts

num_rev_tl_bal_gt_0	Number of revolving trades with balance >0

num_sats	Number of satisfactory accounts

num_tl_120dpd_2m	Number of accounts currently 120 days past due (updated in past 2 months)

num_tl_30dpd	Number of accounts currently 30 days past due (updated in past 2 months)

num_tl_90g_dpd_24m	Number of accounts 90 or more days past due in last 24 months

num_tl_op_past_12m	Number of accounts opened in past 12 months

open_acc	The number of open credit lines in the borrower's credit file

open_acc_6m	Number of open trades in last 6 months

open_act_il	Number of currently active installment trades

open_il_12m	Number of installment accounts opened in past 12 months

open_il_24m	Number of installment accounts opened in past 24 months

open_rv_12m	Number of revolving trades opened in past 12 months

open_rv_24m	Number of revolving trades opened in past 24 months

orig_projected_additional_accrued_interest	The original projected additional interest amount that will accrue for the given hardship payment plan as of the Hardship Start Date. This field will be null if the borrower has broken their hardship payment plan.

out_prncp	Remaining outstanding principal for total amount funded

out_prncp_inv	Remaining outstanding principal for portion of total amount funded by investors

payment_plan_start_date	The day the first hardship plan payment is due. For example, if a borrower has a hardship plan period of 3 months, the start date is the start of the three-month period in which the borrower is allowed to make interest-only payments

pct_tl_nvr_dlq	Percent of trades never delinquent

percent_bc_gt_75	Percentage of all bankcard accounts > 75% of limit

policy_code	"publicly available policy_code=1

new products not publicly available policy_code=2"

pub_rec	Number of derogatory public records

pub_rec_bankruptcies	Number of public record bankruptcies

purpose	A category provided by the borrower for the loan request

pymnt_plan	Indicates if a payment plan has been put in place for the loan

recoveries	post charge off gross recovery

revol_bal	Total credit revolving balance

revol_bal_joint 	 Sum of revolving credit balance of the co-borrowers, net of duplicate balances

revol_util	Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit

sec_app_chargeoff_within_12_mths 	 Number of charge-offs within last 12 months at time of application for the secondary applicant

sec_app_collections_12_mths_ex_med 	 Number of collections within last 12 months excluding medical collections at time of application for the secondary applicant

sec_app_earliest_cr_line 	 Earliest credit line at time of application for the secondary applicant

sec_app_fico_range_high 	 FICO range (low) for the secondary applicant

sec_app_fico_range_low 	 FICO range (high) for the secondary applicant

sec_app_inq_last_6mths 	 Credit inquiries in the last 6 months at time of application for the secondary applicant

sec_app_mort_acc 	 Number of mortgage accounts at time of application for the secondary applicant

sec_app_mths_since_last_major_derog 	 Months since most recent 90-day or worse rating at time of application for the secondary applicant

sec_app_num_rev_accts 	 Number of revolving accounts at time of application for the secondary applicant

sec_app_open_acc 	 Number of open trades at time of application for the secondary applicant

sec_app_open_act_il	 Number of currently active installment trades at time of application for the secondary applicant

sec_app_revol_util 	 Ratio of total current balance to high credit/credit limit for all revolving accounts

settlement_amount	The loan amount that the borrower has agreed to settle for

settlement_date	The date that the borrower agrees to the settlement plan

settlement_percentage	The settlement amount as a percentage of the payoff balance amount on the loan

settlement_status	The status of the borrower’s settlement plan. Possible values are: COMPLETE, ACTIVE, BROKEN, CANCELLED, DENIED, DRAFT

settlement_term	The number of months that the borrower will be on the settlement plan

sub_grade	LC assigned loan subgrade

tax_liens	Number of tax liens

term	The number of payments on the loan. Values are in months and can be either 36 or 60

title	The loan title provided by the borrower

tot_coll_amt	Total collection amounts ever owed

tot_cur_bal	Total current balance of all accounts

tot_hi_cred_lim	Total high credit/credit limit

total_acc	The total number of credit lines currently in the borrower's credit file

total_bal_ex_mort	Total credit balance excluding mortgage

total_bal_il	Total current balance of all installment accounts

total_bc_limit	Total bankcard high credit/credit limit

total_cu_tl	Number of finance trades

total_il_high_credit_limit	Total installment high credit/credit limit

total_pymnt	Payments received to date for total amount funded

total_pymnt_inv	Payments received to date for portion of total amount funded by investors

total_rec_int	Interest received to date

total_rec_late_fee	Late fees received to date

total_rec_prncp	Principal received to date

total_rev_hi_lim  	Total revolving high credit/credit limit

url	URL for the LC page with listing data

verification_status	Indicates if income was verified by LC, not verified, or if the income source was verified

verified_status_joint	Indicates if the co-borrowers' joint income was verified by LC, not verified, or if the income source was verified

zip_code	The first 3 numbers of the zip code provided by the borrower in the loan application

### Displaying data types.

In [6]:
df.dtypes

member_id                                     float64
loan_amnt                                     float64
funded_amnt                                   float64
funded_amnt_inv                               float64
term                                           object
int_rate                                      float64
installment                                   float64
grade                                          object
sub_grade                                      object
emp_title                                      object
emp_length                                     object
home_ownership                                 object
annual_inc                                    float64
verification_status                            object
issue_d                                        object
loan_status                                    object
pymnt_plan                                     object
url                                            object
desc                        

### Counting null values in each column.

In [7]:
(100*df[df.columns[df.isnull().any()]].isnull().sum()/len(df.index)).sort_values()

zip_code                                        0.000074
inq_last_6mths                                  0.000074
tax_liens                                       0.002899
last_credit_pull_d                              0.004088
chargeoff_within_12_mths                        0.004163
collections_12_mths_ex_med                      0.004163
dti                                             0.027800
pub_rec_bankruptcies                            0.051810
revol_util                                      0.063703
last_pymnt_d                                    0.171931
title                                           1.238302
total_bc_limit                                  3.514506
mort_acc                                        3.514506
acc_open_past_24mths                            3.514506
total_bal_ex_mort                               3.514506
num_sats                                        4.150791
num_bc_sats                                     4.150791
mths_since_recent_bc           

In [8]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
member_id,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loan_amnt,1.34531e+06,NaN,NaN,NaN,14420,8717.05,500,8000,12000,20000,40000
funded_amnt,1.34531e+06,NaN,NaN,NaN,14411.6,8713.12,500,8000,12000,20000,40000
funded_amnt_inv,1.34531e+06,NaN,NaN,NaN,14389.1,8715.49,0,7875,12000,20000,40000
term,1345310,2,36 months,1020743,NaN,NaN,NaN,NaN,NaN,NaN,NaN
int_rate,1.34531e+06,NaN,NaN,NaN,13.2396,4.76872,5.31,9.75,12.74,15.99,30.99
installment,1.34531e+06,NaN,NaN,NaN,438.076,261.513,4.93,248.48,375.43,580.73,1719.83
grade,1345310,7,B,392741,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub_grade,1345310,35,C1,85494,NaN,NaN,NaN,NaN,NaN,NaN,NaN
emp_title,1259525,378353,Teacher,21268,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Dropping columns with >90% null values.

In [9]:
df.drop(['member_id','desc','hardship_type','hardship_reason','hardship_status','deferral_term','hardship_amount','hardship_start_date','hardship_end_date','payment_plan_start_date','hardship_length','hardship_dpd','hardship_loan_status','orig_projected_additional_accrued_interest','hardship_payoff_balance_amount','hardship_last_payment_amount','debt_settlement_flag_date','settlement_status','settlement_date','settlement_amount','settlement_percentage','settlement_term'], axis=1,inplace=True)

### Dropping columns that apply to two joint applicants.

In [10]:
df = df[df['application_type']!='Joint App']
df.drop(['revol_bal_joint','dti_joint','verification_status_joint','sec_app_fico_range_low','sec_app_fico_range_high','sec_app_earliest_cr_line','sec_app_inq_last_6mths','sec_app_mort_acc','sec_app_open_acc','sec_app_revol_util','sec_app_open_act_il','sec_app_num_rev_accts','sec_app_chargeoff_within_12_mths','sec_app_collections_12_mths_ex_med','sec_app_mths_since_last_major_derog','annual_inc_joint','application_type'],axis=1,inplace=True)

### Dropping 'funded_amnt' and 'funded_amnt_inv' since they're essentially the same as 'loan_amnt'.

In [11]:
df.drop(['funded_amnt','funded_amnt_inv'], axis=1,inplace=True)

### Removing non-numeric characters from 'emp_length'.  "< 1 year" we'll replace with 0.

In [12]:
df['emp_length'].value_counts()

10+ years    435633
2 years      119867
3 years      106015
< 1 year     102544
1 year        87271
5 years       82927
4 years       79311
6 years       61920
8 years       60004
7 years       58919
9 years       50335
Name: emp_length, dtype: int64

In [13]:
df['emp_length']=df['emp_length'].str.replace('< 1 year','0')
df['emp_length']=pd.to_numeric(df['emp_length'].str.replace('\D',''))

### Removing non-numeric characters from 'term' (loan term length in months).

In [14]:
df['term'].value_counts()

 36 months    1004477
 60 months     315033
Name: term, dtype: int64

In [15]:
df['term']=pd.to_numeric(df['term'].str.replace('\D',''))

### Calculating the distance between 'earliest_cr_line' (date of opening their first line of credit) and 'issue_d' (date the loan was issued) in order to see if it has predictive value.

In [16]:
df['issue_d'] = pd.to_datetime(df['issue_d'],format="%b-%Y")
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'],format="%b-%Y")
df['earliest_to_loan_issue'] = ((df['issue_d'] - df['earliest_cr_line']).dt.days/365.25)

### Dropping variables less relevant to predictions: 'url', 'zip_code', 'addr_state', 'initial_list_status','last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d', 'hardship_flag', 'disbursement_method', 'debt_settlement_flag', 'mths_since_last_delinq', 'mths_since_last_record', 'revol_util', 'mths_since_rcnt_il', 'il_util', 'all_util', 'avg_cur_bal', 'bc_open_to_buy', 'last_pymnt_amnt', 'last_fico_range_high', 'last_fico_range_low', 'bc_util', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'emp_title', 'mths_since_recent_inq', 'pymnt_plan', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'policy_code', 'acc_now_delinq','open_acc_6m','open_rv_12m', 'open_rv_24m', 'acc_open_past_24mths', 'num_actv_rev_tl', 'num_rev_tl_bal_gt_0', 'tot_cur_bal', 'total_bal_il', 'max_bal_bc', 'open_act_il', 'open_il_12m', 'inq_last_6mths',  'num_tl_120dpd_2m', 'num_tl_30dpd', 'inq_fi', 'total_cu_tl'

In [17]:
df.drop((['url', 'zip_code', 'addr_state','initial_list_status','last_pymnt_d', 'next_pymnt_d',
       'last_credit_pull_d', 'hardship_flag', 'disbursement_method', 'debt_settlement_flag', 'mths_since_last_delinq', 'mths_since_last_record', 'revol_util','mths_since_rcnt_il', 'il_util', 'all_util', 'avg_cur_bal', 'bc_open_to_buy', 'last_pymnt_amnt', 'last_fico_range_high', 'last_fico_range_low','bc_util','pct_tl_nvr_dlq','percent_bc_gt_75','emp_title','mths_since_recent_inq','pymnt_plan','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','policy_code','acc_now_delinq','open_acc_6m','open_rv_12m', 'open_rv_24m','acc_open_past_24mths', 'num_actv_rev_tl','num_rev_tl_bal_gt_0','tot_cur_bal','total_bal_il', 'max_bal_bc', 'open_act_il', 'open_il_12m', 'inq_last_6mths',  'num_tl_120dpd_2m', 'num_tl_30dpd', 'inq_fi', 'total_cu_tl']), axis=1,inplace=True)

### Dropping more specific 'sub_grade' in favor of 'grade' and creating dummies.

In [18]:
df.drop(['sub_grade'], axis=1, inplace=True)
df['grade'].value_counts()

B    386373
C    373401
A    231630
D    196278
E     91664
F     31351
G      8813
Name: grade, dtype: int64

### Dropping 'title' as it's a dupe of 'purpose' (reason for applying for a loan).

In [19]:
df.drop('title', axis=1,inplace=True)
df['purpose'].value_counts()

debt_consolidation    764351
credit_card           290840
home_improvement       85456
other                  76429
major_purchase         28931
small_business         15180
medical                15097
car                    14380
moving                  9289
vacation                8934
house                   7085
wedding                 2294
renewable_energy         918
educational              326
Name: purpose, dtype: int64

### Displaying remaining categorical features.

In [20]:
df['home_ownership'].value_counts()

MORTGAGE    648107
RENT        528509
OWN         142418
ANY            286
OTHER          144
NONE            46
Name: home_ownership, dtype: int64

In [21]:
df['verification_status'].value_counts()

Source Verified    513873
Verified           408334
Not Verified       397303
Name: verification_status, dtype: int64

In [22]:
df['loan_status'].value_counts()

Fully Paid     1057295
Charged Off     262215
Name: loan_status, dtype: int64

### Counting null values in each column again.

In [23]:
(100*df[df.columns[df.isnull().any()]].isnull().sum()/len(df.index)).sort_values()

tax_liens                          0.002956
collections_12_mths_ex_med         0.004244
chargeoff_within_12_mths           0.004244
pub_rec_bankruptcies               0.052823
mort_acc                           3.583224
total_bal_ex_mort                  3.583224
total_bc_limit                     3.583224
num_bc_sats                        4.231950
num_sats                           4.231950
mths_since_recent_bc               4.519329
num_op_rev_tl                      5.117582
num_il_tl                          5.117582
num_bc_tl                          5.117582
num_actv_bc_tl                     5.117582
num_accts_ever_120_pd              5.117582
total_il_high_credit_limit         5.117582
mo_sin_rcnt_tl                     5.117582
total_rev_hi_lim                   5.117582
tot_hi_cred_lim                    5.117582
tot_coll_amt                       5.117582
num_tl_op_past_12m                 5.117582
num_tl_90g_dpd_24m                 5.117582
mo_sin_rcnt_rev_tl_op           

### Dropping the 5.1% of rows that are all missing the same features.

In [24]:
df.dropna(subset=['num_rev_accts', 'mo_sin_rcnt_rev_tl_op'], inplace=True)
(100*df[df.columns[df.isnull().any()]].isnull().sum()/len(df.index)).sort_values()

mths_since_recent_bc               0.967427
mo_sin_old_il_acct                 2.978240
emp_length                         5.809673
open_il_24m                       59.098980
inq_last_12m                      59.099060
mths_since_recent_revol_delinq    65.114087
mths_since_last_major_derog       72.235361
mths_since_recent_bc_dlq          74.962080
dtype: float64

### Filling row where 'dti' == -1 with median.

In [25]:
df['dti'][df['dti'] < 0] = df['emp_length'].median()

### Filling these features with 0 as they measure events that are possible to not occur within these short time frames.
#### 'open_il_24m' - Number of installment accounts opened in past 2 years.
#### 'inq_last_12m' - Number of credit inquiries in past year.

In [26]:
df['open_il_24m'].fillna(value=0, inplace=True)
df['inq_last_12m'].fillna(value=0, inplace=True)

### Filling these features with max since they measure the length of time since the last occurence of a bad credit event happening, and it's possible that these events have never happened.
#### 'mths_since_last_major_derog' - Months since most recent 90-day or worse rating.
#### 'mths_since_recent_bc_dlq' - Months since most recent bankcard delinquency.
#### 'mths_since_recent_revol_delinq' - Months since most recent revolving delinquency.

In [27]:
df['mths_since_last_major_derog'].fillna(df['mths_since_last_major_derog'].max(),inplace=True)
df['mths_since_recent_bc_dlq'].fillna(df['mths_since_recent_bc_dlq'].max(),inplace=True)
df['mths_since_recent_revol_delinq'].fillna(df['mths_since_recent_revol_delinq'].max(),inplace=True)

### Filling these features with median since there are few null values, and there's no intuitive way to interpret them. 
#### 'emp_length' - Current employment length in years.
#### 'mths_since_recent_bc' - Months since most recent bankcard delinquency.
#### 'mo_sin_old_il_acct' - Months since oldest bank installment account opened.

In [28]:
df['emp_length'].fillna(df['emp_length'].median(),inplace=True)
df['mths_since_recent_bc'].fillna(df['mths_since_recent_bc'].median(),inplace=True)
df['mo_sin_old_il_acct'].fillna(df['mo_sin_old_il_acct'].median(),inplace=True)

### No null values left.

In [29]:
df[df.columns[df.isnull().any()]].isnull().sum().sort_values()

Series([], dtype: float64)

### We'll create dummies for the categorical features.

In [30]:
dummies = pd.get_dummies(data=df, columns=['grade','purpose','home_ownership','verification_status', 'loan_status'], prefix=['gra','pur','hom_own','ver_sta','loan_sta'], dtype=int)

## Exporting dataframe as CSV

In [31]:
df.to_csv( "df2.csv", encoding='utf-8', index=True)
dummies.to_csv( "dummies2.csv", encoding='utf-8', index=True)